# Bring your own model to create a Active Learning Workflow

### Using this Notebook

Please set the image to *TensorFlow 2.10.0  Python 3.9 CPU Optimized* and kernel to  *Python 3*  when running this notebook .Select instance type as `ml.t3.medium`


This notebook is comprised of two parts , In Part 1, we show you how you can containerize your own Machine Learning model and push it to [Amazon Elastic Container Registry (ECR)](https://docs.aws.amazon.com/AmazonECR/latest/userguide/what-is-ecr.html). This notebook will produce an Image ID that you can use to integrate your model into an active learning workflow. In this part, we will leverage sagemaker studio image build  which is a convenience package that allows data scientists and developers to easily build custom container images from your Studio notebooks via a new [CLI](https://github.com/aws-samples/sagemaker-studio-image-build-cli) . The new CLI eliminates the need to manually set up and connect to Docker build environments for building container images in Amazon SageMaker Studio. 

We will also leverage leverage [AWS SAM](https://aws.amazon.com/serverless/sam/)  to manage and deploy AWS Cloudformation templates that will be used  for Active Labeling workflow. 

In the Part 2, we will use this model to create an automated labeling workflow for a text-classification labeling job

###  Part 1:  Build you own ML model and push to ECR

Install sagemaker studio image build  and AWS SAM to manage and deploy AWS Cloudformation template

In [ ]:
%pip install --upgrade setuptools  sagemaker-studio-image-build aws-sam-cli

Restart your kernel for this notebook to use updated packages 

In [ ]:
%%html

<p><b>Restart your kernel for this notebook to use update package.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:restart" style="display:none;">Restart Kernel</button>

<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>


Verify sagemaker-studio-image-build and aws-sam-cli

In [ ]:
%%bash

sam  --version
sm-docker --version


#### Setup SageMaker environment and variables

In [ ]:
import os, sys, sagemaker, tensorflow as tf, pandas as pd, boto3, numpy as np
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

sess = sagemaker.Session()

role = get_execution_role()
region = sess.boto_session.region_name
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
smclient = boto3.Session().client('sagemaker')
bucket = sess.default_bucket()
stack_name= "active-learning-stack"
key = "sagemaker-byoal"

We use this notebook to tokenize our dataset and create a training dataset, add a containerized model to ECR, and train the model. The notebook will produce an image name in ECR which can be used for training and inference across Amazon SageMaker. 

We use a Keras deep learning model for demonstration purposes only. The methodology for developing and containerizing our model was inspired by the tutorial [Take an ML from idea to production using Amazon SageMaker](https://github.com/aws-samples/amazon-sagemaker-keras-text-classification) and is not included in the notebook. 

#### Preprocessing and Tokenizing the data

First we will download the dataset, read the csv news dataset using pandas and clean the data:

* We make all alphanumeric characters lowercase and replace undesired characters. 
* We remove stop words and empty records. 

The result is saved into a CSV formatted file.



In [ ]:
! wget -nc https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip --no-check-certificate && unzip -o NewsAggregatorDataset.zip

Select the column names required from the downloaded dataset

In [ ]:
column_names = ["TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"]
manifest_file = "partially-labeled.manifest"
news_data_all = pd.read_csv("newsCorpora.csv", names=column_names, header=None, delimiter="\t")
news_data = news_data_all.sample(n=10000, random_state=42)
news_data = news_data[["TITLE", "CATEGORY"]]
news_data 

The below cell will clean our data set using pandas.

In [ ]:
news_data["TITLE"].replace('"', "", inplace=True, regex=True)
news_data["TITLE"].replace("[^\w\s]", "", inplace=True, regex=True)
news_data["TITLE"] = news_data["TITLE"].str.split("\n").str[0]
news_data["CATEGORY"] = news_data["CATEGORY"].astype("category").cat.codes
news_data

In [ ]:
fixed = news_data["TITLE"].str.lower().replace('"', "")

The below cell creates the `news_subset.csv` which is used to manifest file for training

In [ ]:
news_data.to_csv("news_subset.csv", index=False)

Next, we use the [Keras Tokenizer class](https://keras.io/preprocessing/text/) to tokenize our dataset and upload it to S3. 

In [ ]:
import os, pickle

from sklearn.feature_extraction import _stop_words

stop_words = _stop_words.ENGLISH_STOP_WORDS
import os, sys, sagemaker, tensorflow as tf, pandas as pd, boto3, numpy as np

train_s3_key = "sagemaker/news_subset.csv"
boto3.resource("s3").Bucket(bucket).upload_file("news_subset.csv", train_s3_key)

column_names = ["TITLE", "CATEGORY"]
tf_train = pd.read_csv(
    "news_subset.csv", names=column_names, header=None, skiprows=[0], delimiter=","
)
tf_train = tf_train[column_names]

tf_train["TITLE"] = tf_train["TITLE"].str.lower().replace("[^\w\s]", "")
tf_train["TITLE"] = tf_train["TITLE"].apply(
    lambda x: " ".join([word for word in x.split() if word not in (stop_words)])
)
tf_train.dropna(inplace=True)

cat = tf_train["CATEGORY"].astype("category").cat.categories
tf_train["CATEGORY"] = tf_train["CATEGORY"].astype("category").cat.codes
y = tf_train["CATEGORY"].values


max_features = 5000  # we set maximum number of words to 5000
maxlen = 100  # and maximum sequence length to 100
embedding_dim = 50  # this is the final dimension of the embedding space.
tok = tf.keras.preprocessing.text.Tokenizer(num_words=max_features)  # tokenizer step
tok.fit_on_texts(list(tf_train["TITLE"]))  # fit to cleaned text
with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)
boto3.resource("s3").Bucket(bucket).upload_file("tokenizer.pickle", key + "/tokenizer.pickle")

The below cell will update the value of *tokenizer_bucket* in our training and prediction scripts within the container.

In [ ]:
def inplace_string_replace(filename, old_string, new_string):
    with open(filename) as f:
        updated_text = f.read().replace(old_string, new_string)

    with open(filename, "w") as f:
        f.write(updated_text)


old_code = "tokenizer_bucket = '<Update tokenizer bucket here>'"
new_code = "tokenizer_bucket = '{}'".format(bucket)
inplace_string_replace("./container/news-classifier/train", old_code, new_code)
inplace_string_replace("./container/news-classifier/predictor.py", old_code, new_code)

We extract the first 1000 entries for training and add them to a manifest file. Then, we save our training manifest file in S3. 

In [ ]:
column_names = ["TITLE", "CATEGORY"]

tf_train = pd.read_csv(
    "news_subset.csv", names=column_names, header=None, skiprows=[0], delimiter=","
)
tf_train = tf_train[["TITLE", "CATEGORY"]]
tf_train["TITLE"] = tf_train["TITLE"].str.replace('"', "").replace("\r", "")
tf_train["CATEGORY"] = tf_train["CATEGORY"].astype("category").cat.codes

val_file = "validation-manifest"
series = pd.Series(
    data=tf_train.iloc[:1000].TITLE.values, index=tf_train.iloc[:1000].CATEGORY.values
)
with open(val_file, "w") as outfile:
    for items in series.items():
        outfile.write('{"category":' + str(items[0]) + ',"source":"' + items[1] + '"}\n')
boto3.resource("s3").Bucket(bucket).upload_file(val_file, key + "/" + val_file)
valdiate_s3_uri = "s3://{}/{}".format(bucket, key + "/" + val_file)

train_file = "train-manifest"
series = pd.Series(
    data=tf_train.iloc[1000:7000].TITLE.values, index=tf_train.iloc[1000:7000].CATEGORY.values
)
with open(train_file, "w") as outfile:
    for items in series.items():
        outfile.write('{"category":' + str(items[0]) + ',"source":"' + items[1] + '"}\n')

boto3.resource("s3").Bucket(bucket).upload_file(train_file, key + "/" + train_file)
train_s3_uri = "s3://{}/{}".format(bucket, key + "/" + train_file)

#### Adding the Containerized ML Model to ECR

Setup ECR repoisitory name and image version 

In [ ]:
repo_name="news-classifier"
image_version="1.0"

The next cell will build a Amazon SageMaker-compatible Docker images directly from your Amazon SageMaker Studio environment.The CLI abstracts the previous need to set up a secondary build environment and allows you to focus and spend time on the ML problem you’re trying to solve as opposed to creating workflows for Docker builds. The new CLI automatically sets up your reusable build environment that you interact with via high-level commands. You essentially tell the CLI to build your image, without having to worry about the underlying workflow orchestrated through the CLI, and the output is a link to your Amazon Elastic Container Registry (Amazon ECR) image location

#### Build docker container and push it to ECR

Image URI is printed at the end of successful completion of SM Docker build command . This command takes approximately 4 mins to get completed

In [ ]:
 %%bash -s  "$repo_name"  "$image_version"
    
cd container

sm-docker build  .  --repository  ${1}:${2}

####  Training our Model

We train our model on the training data that we extracted above and see the accuracy returned by our algorithm in Amazon SageMaker.

In [ ]:
from sagemaker.estimator import Estimator

image_uri="{0}.dkr.ecr.{1}.amazonaws.com/{2}:{3}".format(account, region, repo_name, image_version)
print(image_uri)

estimator = Estimator(
    image_uri=image_uri, role=role, instance_count=1, instance_type="ml.m5.xlarge"
)

estimator.fit({"training": train_s3_uri, "validation": valdiate_s3_uri})


### Part 2: Use  model for Active Labeling workflow 

Update template.yaml . The template.yaml is leverage by AWS SAM to build out the backend infrastructure consisting of AWS step functions, helper utiltiies and AWS lambda functions

Update configurations variables for Active Learning process  :

- `byom` : (Bring your own model)  . This parameter determines whether the Lambda function used for configuring the training parameters returns the Amazon Blazing Text algorithm or your own custom model docker image  . This is set to to `true` because we are using our own custom ML model 
- `byomimage` : Your own custom model image
- `sagemaker_program` :  This parameter determines the training script to be used for training your model . In this case we are using our own custom built docker image that contains the training script . Hence, setting this paameter to NotApplicable
- `sagemaker_submit_directory` : The directory which contains the training source code or other dependencies aside from the entry point file . In this case we are using our own custom built docker image that contains the training script and other dependencies . Hence, setting this parameter to NotApplicable
- `pretrain_model` : This parameter determines if the active learning process is using pretrained model or not   . This is set to to `false` because we are using our own custom ML model .
- `pretrain_algo_train_repo` :  This parameter refers to the training image URI of our  pretrained BERT algorithm . Because we are not using a pretrained BERT model, setting this parameter to NotApplicable
- `base_model_uri` : The URI of the pretrained BERT model which will be fine tuned with propriatary data. Because we are not using pretrained models, setting this parameter to NotApplicable
- `pretrain_algo_inference_repo` :  This parameter refers to th inference image URI of our  pretrained BERT algorithm . Because we are not using a pretrained BERT model, setting this parameter to NotApplicable


Update Batch Strategy to `Single` in the `CreateTransformJob` step of the ActiveLearning-active-learning-stack state machine.
This is because the TensorFlow algorithm used here supports  `SingleRecord` 
- `BatchStrategy `: `SingleRecord`

In [ ]:
%%bash -s  "$image_uri"
cd src

# Update byom value to true 
sed  -i  "0,/^\([[:space:]]*byom: *\).*/s//\1"true"/"  template.yaml

#Update byomimage value to your own custom ML image
sed  -i  "0,/^\([[:space:]]*byomimage: *\).*/s//\1${1//\//\\\/}/"  template.yaml

#Update Training script for PreTrained model to  Not Applicable
sed  -i  "0,/^\([[:space:]]*sagemaker_program: *\).*/s//\1"NotApplicable"/"   template.yaml

#Update Pretrained Model flag to False
sed  -i  "0,/^\([[:space:]]*pretrain_model: *\).*/s//\1"false"/"   template.yaml

#Update Pretrained algorithm training repository to NotApplicable
sed  -i  "0,/^\([[:space:]]*pretrain_algo_train_repo: *\).*/s//\1"NotApplicable"/"  template.yaml

#Update SageMaker Submit Directory to NotApplicable
sed  -i  "0,/^\([[:space:]]*sagemaker_submit_directory: *\).*/s//\1"NotApplicable"/"   template.yaml

#Update base pretrained model URI to NotApplicable
sed  -i  "0,/^\([[:space:]]*base_model_uri: *\).*/s//\1"NotApplicable"/"   template.yaml

#Updated Pretrained  algorithm inference  reposiistory  to NotApplicable
sed  -i  "0,/^\([[:space:]]*pretrain_algo_inference_repo: *\).*/s//\1"NotApplicable"/"   template.yaml

#Update BatchStrategy to SingleRecord
sed -i 's/MultiRecord/SingleRecord/g'  template.yaml

cat template.yaml

Use `sam build` to update the AWS CloudFormation template and `sam deploy` to deploy the AWS CLouformation change set

In [ ]:
%%bash -s  "$region"  "$stack_name"  "$bucket"

# Change directory to src folder
cd src

#Delete SAM build folder to clean up any exisiting SAM templates
rm  -rf  .aws-sam

#Build SAM artifacts to deploy AWS Cloudformation template
sam build

#Deploy SAM template
sam deploy  --region ${1}  --stack-name ${2}  --s3-bucket ${3}  --capabilities CAPABILITY_IAM CAPABILITY_AUTO_EXPAND



#### Prepare labeling input manifest file

We will create an input manifest file for our active learning workflow using the newsCorpora.csv file from the [UCI News Dataset](https://archive.ics.uci.edu/ml/datasets/News+Aggregator). This dataset contains a list of about 420,000 articles that fall into one of four categories: Business (b), Science & Technology (t), Entertainment (e) and Health & Medicine (m). We will randomly choose 10,000 articles from that file to create our dataset.

For the active learning loop to start, 20% of the data must be labeled. To quickly test the active learning component, we will include 20% (`labeled_count`) of the original labels provided in the dataset in our input manifest. We use this partially-labeled dataset as the input to the active learning loop.

The following cell will create our partially-labeled input manifest file, and push it to our S3 bucket. 

In [ ]:
import json

total = len(news_data)
labeled_count = int(total / 5)  # 20% of the dataset is labeled.
label_map = {
    "b": "Business",
    "e": "Entertainment",
    "m": "Health & Medicine",
    "t": "Science and Technology",
}
labeled_series = pd.Series(
    data=news_data.iloc[:labeled_count].TITLE.values,
    index=news_data.iloc[:labeled_count].CATEGORY.values,
)
annotation_metadata = b"""{ "category-metadata" : { "confidence": 1.0, "human-annotated": "yes", "type": "groundtruth/text-classification"} }"""
annotation_metadata_dict = json.loads(annotation_metadata)
with open(manifest_file, "w") as outfile:
    for items in labeled_series.items():
        labeled_record = dict()
        labeled_record["source"] = items[1]
        labeled_record["category"] = int(items[0])
        labeled_record.update(annotation_metadata_dict)
        outfile.write(json.dumps(labeled_record) + "\n")

unlabeled_series = pd.Series(
    data=news_data.iloc[labeled_count:].TITLE.values,
    index=news_data.iloc[labeled_count:].CATEGORY.values,
)
with open(manifest_file, "a") as outfile:
    for items in unlabeled_series.items():
        outfile.write('{"source":"' + items[1] + '"}\n')

boto3.resource("s3").Bucket(bucket).upload_file(manifest_file, key + "/" + manifest_file)
manifest_file_uri = "s3://{}/{}".format(bucket, key + "/" + manifest_file)

In [ ]:
# Use s3 client to upload relevant json strings to s3.
s3_client = boto3.client("s3")

This cell will specify the labels that workers will use to categorize the articles. To customize your labeling job, add your own labels here. To learn more, see [LabelCategoryConfigS3Uri](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateLabelingJob.html#sagemaker-CreateLabelingJob-request-LabelCategoryConfigS3Uri).

In [ ]:
label_file_name = "class_labels.json"
label_file = """{
    "document-version": "2018-11-28",
    "labels": [
        {
            "label": "Business"
        },
        {
            "label": "Entertainment"
        },
        {
            "label": "Health & Medicine"
        },
        {
            "label": "Science and Technology"
        }
    ]
}"""

s3_client.put_object(Body=label_file, Bucket=bucket, Key=key + "/" + label_file_name)
label_file_uri = "s3://{}/{}".format(bucket, key + "/" + label_file_name)

The following cell will specify our custom worker task template. This template will configure the UI that workers will see when they open our text classification labeling job tasks. To learn how to customize this cell, see  [Creating your custom labeling task template](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-custom-templates-step2.html).

In [ ]:
template_file_name = "instructions.template"
template_file = r"""
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>
<crowd-form>
  <crowd-classifier
    name="crowd-classifier"
    categories="{{ task.input.labels | to_json | escape }}"
    header="Select the news title corresponding to the 4 categories. (b) for Business, (e) for Entertainment, (m) for Health and Medicine and (t) for Science and Technology."
  >
    <classification-target> {{ task.input.taskObject }} </classification-target>
    <full-instructions header="Classifier instructions">
      <ol><li><strong>Read</strong> the text carefully.</li><li><strong>Read</strong> the examples to understand more about the options.</li><li><strong>Choose</strong> the appropriate label that best suits the text.</li></ol>
    </full-instructions>
    <short-instructions>
      <p>Example Business title:</p><p>US open: Stocks fall after Fed official hints at accelerated tapering.</p><p><br>
      </p><p>Example Entertainment title:</p><p>CBS negotiates three more seasons for The Big Bang Theory</p><p><br>
      </p><p>Example Health & Medicine title:</p><p>Blood Test Could Predict Alzheimer's. Good News? </p><p><br>
      </p><p>Example Science and Technology (t) title:</p><p>Elephants tell human friend from foe by voice.</p><p><br>
      </p>
    </short-instructions>
  </crowd-classifier>
</crowd-form>
"""

s3_client.put_object(Body=template_file, Bucket=bucket, Key=key + "/" + template_file_name)
template_file_uri = "s3://{}/{}".format(bucket, key + "/" + template_file_name)

To use a private work team to labeling your data objects, set `USE_PRIVATE_WORKFORCE` to `True` and input your work team ARN for `private_workteam_arn`. You must have a private workforce in the same AWS Region as your labeling job task request to use a private work team. To learn more see [Use a Private Workforce](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-workforce-private.html)

In [ ]:
USE_PRIVATE_WORKFORCE = True
private_workteam_arn = "Enter your Private Worker ARN"

The below cell will automatically configure a public workforce ARN and pre- and post-annotation ARNs (`prehuman_arn` and `acs_arn` respectively). If `USE_PRIVATE_WORKFORCE` is `False` a public workforce will be used to create your labeling job request. 

To customize your labeling job task type, you will need to modify `prehuman_arn` and `acs_arn`. 

If you are using one of the Ground Truth built-in task types, you can find pre- and post-annotation lambda ARNs using the following links. 
* Pre-annotation lambda ARNs for built in task types can be found in [HumanTaskConfig](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_HumanTaskConfig.html#API_HumanTaskConfig_Contents).
* Post-annotation lambda ARNs (Annotation Consolidation Lambda) for built in task types can be found in [AnnotationConsolidationConfig](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_AnnotationConsolidationConfig.html#sagemaker-Type-AnnotationConsolidationConfig-AnnotationConsolidationLambdaArn).

If you are creating a custom labeling job task, see [Step 3: Processing with AWS Lambda
](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-custom-templates-step3.html) learn how to create custom pre- and post-annotation lambda ARNs.

In [ ]:
# Specify ARNs for resources needed to run a text classification job.
ac_arn_map = {
    "us-west-2": "081040173940",
    "us-east-1": "432418664414",
    "us-east-2": "266458841044",
    "eu-west-1": "568282634449",
    "ap-northeast-1": "477331159723",
}

public_workteam_arn = "arn:aws:sagemaker:{}:394669845002:workteam/public-crowd/default".format(
    region
)
prehuman_arn = "arn:aws:lambda:{}:{}:function:PRE-TextMultiClass".format(region, ac_arn_map[region])
acs_arn = "arn:aws:lambda:{}:{}:function:ACS-TextMultiClass".format(region, ac_arn_map[region])

The following cell specifies our labeling job name, the description workers see, and tags that workers can use to find our labeling job task.

In [ ]:
job_name_prefix = "byoal-news"
task_description = "Classify news title to one of these 4 categories."
task_keywords = ["text", "classification", "humans", "news"]
task_title = task_description

Modify the following request to customize your labeling job request. For more information on the parameters below, see [CreateLabelingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateLabelingJob.html).

In [ ]:
human_task_config = {
    "AnnotationConsolidationConfig": {
        "AnnotationConsolidationLambdaArn": acs_arn,
    },
    "PreHumanTaskLambdaArn": prehuman_arn,
    "MaxConcurrentTaskCount": 10,  # 10 texts will be sent at a time to the workteam.
    "NumberOfHumanWorkersPerDataObject": 1,  # 1 workers will be enough to label each text.
    "TaskAvailabilityLifetimeInSeconds": 21600,  # Your work team has 6 hours to complete all pending tasks.
    "TaskDescription": task_description,
    "TaskKeywords": task_keywords,
    "TaskTimeLimitInSeconds": 300,  # Each text must be labeled within 5 minutes.
    "TaskTitle": task_title,
    "UiConfig": {
        "UiTemplateS3Uri": template_file_uri,
    },
}

if not USE_PRIVATE_WORKFORCE:
    human_task_config["PublicWorkforceTaskPrice"] = {
        "AmountInUsd": {
            "Dollars": 0,
            "Cents": 1,
            "TenthFractionsOfACent": 2,
        }
    }
    human_task_config["WorkteamArn"] = public_workteam_arn
else:
    human_task_config["WorkteamArn"] = private_workteam_arn

ground_truth_request = {
    "InputConfig": {
        "DataSource": {
            "S3DataSource": {
                "ManifestS3Uri": manifest_file_uri,
            }
        },
        "DataAttributes": {
            "ContentClassifiers": ["FreeOfPersonallyIdentifiableInformation", "FreeOfAdultContent"]
        },
    },
    "OutputConfig": {
        "S3OutputPath": "s3://{}/{}/output/".format(bucket, key),
    },
    "HumanTaskConfig": human_task_config,
    "LabelingJobNamePrefix": job_name_prefix,
    "RoleArn": role,
    "LabelAttributeName": "category",
    "LabelCategoryConfigS3Uri": label_file_uri,
}

In [ ]:
print(json.dumps(ground_truth_request, indent=2))
step_func_input=json.dumps(ground_truth_request,default=str)

In [ ]:
#Get Stepfunction Client
sfn=boto3.client('stepfunctions', region_name=region)
stateMachineArn="arn:aws:states:{}:{}:stateMachine:ActiveLearningLoop-active-learning-stack".format(region,account)
response = sfn.start_execution(
    stateMachineArn=stateMachineArn,
    input=step_func_input
    
)
executionArn=response["executionArn"] 

print(json.dumps(response, indent=2, default=str))

Please refer to lab instructions for next steps on View active data labeling workflow and Human Annotation WorkFlow

On successful completion of the active learning loop, the state machine will output the final output manifest file and the latest trained model output.

In [ ]:
# Clean up 

# Stop Active Learning state machine
if (sfn.describe_execution(executionArn=executionArn)["status"]=="RUNNING" ):
    sfn.stop_execution(executionArn=executionArn)
    print('State Machine Execution Stopped :  ' + executionArn)

# Stop Ground Truth Labeling jobs that are InProgress or Initializing
response = smclient .list_labeling_jobs(NameContains='byoal-news')
for i in range(len(response['LabelingJobSummaryList'])):
    if ( (response['LabelingJobSummaryList'][i]['LabelingJobStatus'] == 'InProgress' ) or  (response['LabelingJobSummaryList'][i]['LabelingJobStatus'] == 'Initializing')):
        label_job_name=response['LabelingJobSummaryList'][i]['LabelingJobName']
        smclient.stop_labeling_job(LabelingJobName=label_job_name)
        print('Stopping Labeling Job:  ' + label_job_name )

In [ ]:
# Delete AWS ECR Repo -
!aws ecr delete-repository --repository-name news-classifier --force

In [ ]:
%%bash -s  "$region"  "$stack_name"

#Delete CloudFormation Stack 
sam delete  --region ${1}  --stack-name ${2}  --no-prompts